# Building Strategy for Spark Funds’ Business Insvestment

#### Importing all Libraries and Tools for our task

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from urllib.request import urlretrieve
import ssl

#import pycountry
#ISO-8859-1
#ISO 3166-1 country codes

#### Importing the Companies Data to Data Frame named companies from the path 

Encode 'permalink' in comapnies dataframe and decode to ASCII

In [3]:
path = 'https://cdn.upgrad.com/UpGrad/temp/d934844e-5182-4b58-b896-4ba2a499aa57/companies.txt'
companies= pd.read_csv(path, delimiter="\t",encoding='ISO-8859-1')
companies['permalink'] = companies.permalink.str.encode('ISO-8859-1').str.decode('ascii', 'ignore')

#### Importing the Rounds2 Data to Data Frame named rounds2 from the ./_.csv path 

Encode 'company_permalink' in comapnies dataframe and decode to ASCII

In [4]:
rounds2 = pd.read_csv('rounds2.csv',encoding='ISO-8859-1')
rounds2['company_permalink'] = rounds2.company_permalink.str.encode('ISO-8859-1').str.decode('ascii', 'ignore')

#### Visualize both the dataframes

In [5]:
companies.head()

,permalink,name,homepage_url,category_list,status,country_code,state_code,region,city,founded_at
0,/Organization/-Fame,#fame,http://livfame.com,Media,operating,IND,16,Mumbai,Mumbai,NaN
1,/Organization/-Qounter,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,operating,USA,DE,DE - Other,Delaware City,04-09-2014
2,/Organization/-The-One-Of-Them-Inc-,"(THE) ONE of THEM,Inc.",http://oneofthem.jp,Apps|Games|Mobile,operating,NaN,NaN,NaN,NaN,NaN
3,/Organization/0-6-Com,0-6.com,http://www.0-6.com,Curated Web,operating,CHN,22,Beijing,Beijing,01-01-2007
4,/Organization/004-Technologies,004 Technologies,http://004gmbh.de/en/004-interact,Software,operating,USA,IL,"Springfield, Illinois",Champaign,01-01-2010


In [6]:
rounds2.head()

,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd
0,/organization/-fame,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,05-01-2015,10000000.0
1,/ORGANIZATION/-QOUNTER,/funding-round/22dacff496eb7acb2b901dec1dfe5633,venture,A,14-10-2014,NaN
2,/organization/-qounter,/funding-round/b44fbb94153f6cdef13083530bb48030,seed,NaN,01-03-2014,700000.0
3,/ORGANIZATION/-THE-ONE-OF-THEM-INC-,/funding-round/650b8f704416801069bb178a1418776b,venture,B,30-01-2014,3406878.0
4,/organization/0-6-com,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,19-03-2008,2000000.0


#### By Vizualization we can see that both dataframes have companies['permalink'] and rounds2['company_permalink'] in common

>  _Inorder to avoid any mistakes/errors on our primary key we convert them to Lower Case using the 'str' lower command in a vectrozied manner_

In [7]:
companies.permalink = companies.permalink.str.lower()

rounds2.company_permalink = rounds2.company_permalink.str.lower()

#### We are curious to know how many unique companies are there in our company dataframe

In [8]:
company_unique_permalink = np.unique(companies.permalink)
len(company_unique_permalink)

66368

#### Let us see how many unique companies are in our rounds2 dataframe

In [9]:
rounds2_unique_permalink = np.unique(rounds2.company_permalink)

len(rounds2_unique_permalink)

66368

#### For both dataframes we got the same number of unique companies i.e. 66368

Are there any companies in the rounds2 file which are not present in companies?

In [10]:
len(set(rounds2_unique_permalink).difference(set(company_unique_permalink)))

0

> Note: the usage of $sum(~np.isin(rounds2['company permalink'],companies['permalink']))$ is time consuming in its operation and its better to use scalar sets concept here

### Merging the rounds2 frame with companies frame on our Primary Key and creating a Master Frame i.e 'master_frame1'

In [11]:
master_frame1 = pd.merge(rounds2, companies, left_on=rounds2.company_permalink, right_on= companies.permalink)

In [12]:
master_frame1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 114949 entries, 0 to 114948
Data columns (total 16 columns):
company_permalink          114949 non-null object
funding_round_permalink    114949 non-null object
funding_round_type         114949 non-null object
funding_round_code         31140 non-null object
funded_at                  114949 non-null object
raised_amount_usd          94959 non-null float64
permalink                  114949 non-null object
name                       114948 non-null object
homepage_url               108815 non-null object
category_list              111539 non-null object
status                     114949 non-null object
country_code               106271 non-null object
state_code                 104003 non-null object
region                     104782 non-null object
city                       104785 non-null object
founded_at                 94428 non-null object
dtypes: float64(1), object(15)
memory usage: 14.9+ MB


> The Key player here is the __'raised_amount_usd'__ but there are clear nulls in the column $(1 - 954949/114949) * 100 = appx.17.4$

> NOTE: It is not a smart step to remove the NULL value rows which accounts for 17.4% as we observe __'MISSING NOT AT RANDOM PATTERN'__ nor there is a 'Guarantee' of a better result if we impute by mean or median. So, lets retain the DATA

#### Create a dataframe by grouping all the 'funding_round_type' calculate the mean and sort by desending order

In [13]:
t =master_frame1.groupby(master_frame1.funding_round_type).mean()

t.sort_values(by = 'raised_amount_usd',ascending= False)

,raised_amount_usd
funding_round_type,
post_ipo_debt,1.687046e+08
post_ipo_equity,8.218249e+07
secondary_market,7.964963e+07
private_equity,7.330859e+07
undisclosed,1.924237e+07
debt_financing,1.704353e+07
venture,1.174895e+07
grant,4.300576e+06
convertible_note,1.453439e+06


> #### As we know that Spark Funds are intrested in certain window of Investment. Filter the dataframe by considering the avg. funding amount between 5 Million to 15 Million 

In [14]:
t.raised_amount_usd[(t.raised_amount_usd > 5.0e+6) & (t.raised_amount_usd < 1.5e+7)]

funding_round_type
venture    1.174895e+07
Name: raised_amount_usd, dtype: float64

> #### As we know that Spark Funds are intrested in Countries where English is Official language we have a filter here

In [15]:
c = master_frame1.groupby(master_frame1.country_code).sum()

c.sort_values(by = 'raised_amount_usd',ascending= False)[:5]

,raised_amount_usd
country_code,
USA,6.694821e+11
CHN,7.570357e+10
GBR,3.276705e+10
IND,2.768634e+10
CAN,1.842468e+10


> > ### _Note: The conclusion we draw till here in our Analysis is : <br> The most suited Investment Type for Spark Funds in 'Venture' and The Top Three countries of there Interest is 'USA','Great Britan','India'._

#### Creating a new column primary sector from the first entity of 'category_list'

In [16]:
exp = master_frame1.category_list.str.split('|',expand= True)

master_frame1['primary_sector'] = exp[0]

master_frame1.head()

,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd,permalink,name,homepage_url,category_list,status,country_code,state_code,region,city,founded_at,primary_sector
0,/organization/-fame,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,05-01-2015,10000000.0,/organization/-fame,#fame,http://livfame.com,Media,operating,IND,16,Mumbai,Mumbai,NaN,Media
1,/organization/-qounter,/funding-round/22dacff496eb7acb2b901dec1dfe5633,venture,A,14-10-2014,NaN,/organization/-qounter,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,operating,USA,DE,DE - Other,Delaware City,04-09-2014,Application Platforms
2,/organization/-qounter,/funding-round/b44fbb94153f6cdef13083530bb48030,seed,NaN,01-03-2014,700000.0,/organization/-qounter,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,operating,USA,DE,DE - Other,Delaware City,04-09-2014,Application Platforms
3,/organization/-the-one-of-them-inc-,/funding-round/650b8f704416801069bb178a1418776b,venture,B,30-01-2014,3406878.0,/organization/-the-one-of-them-inc-,"(THE) ONE of THEM,Inc.",http://oneofthem.jp,Apps|Games|Mobile,operating,NaN,NaN,NaN,NaN,NaN,Apps
4,/organization/0-6-com,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,19-03-2008,2000000.0,/organization/0-6-com,0-6.com,http://www.0-6.com,Curated Web,operating,CHN,22,Beijing,Beijing,01-01-2007,Curated Web


#### Importing the Rounds2 Data to Data Frame named rounds from the ./_.csv path 

Encode 'category_list' in comapnies dataframe and decode to ASCII

In [17]:
mapping = pd.read_csv('mapping.csv',encoding='ISO-8859-1')
mapping.category_list = mapping.category_list.str.encode('ISO-8859-1').str.decode('ascii', 'ignore')

#### Let us have a quick look at mapping dataframe

In [18]:
mapping.head(25)

,category_list,Automotive & Sports,Blanks,Cleantech / Semiconductors,Entertainment,Health,Manufacturing,"News, Search and Messaging",Others,"Social, Finance, Analytics, Advertising"
0,NaN,0,1,0,0,0,0,0,0,0
1,3D,0,0,0,0,0,1,0,0,0
2,3D Printing,0,0,0,0,0,1,0,0,0
3,3D Technology,0,0,0,0,0,1,0,0,0
4,Accounting,0,0,0,0,0,0,0,0,1
5,Active Lifestyle,0,0,0,0,1,0,0,0,0
6,Ad Targeting,0,0,0,0,0,0,0,0,1
7,Advanced Materials,0,0,0,0,0,1,0,0,0
8,Adventure Travel,1,0,0,0,0,0,0,0,0
9,Advertising,0,0,0,0,0,0,0,0,1


> NOTE: There is an entry 0 in place of 'na' which is observed in index.no 22 i.e 'A0lytics' lets us fix this and wrangle the Data

In [19]:
mapping.category_list = mapping.category_list.str.replace('0','na')

mapping.category_list.loc[mapping.category_list == 'Enterprise 2.na'] = 'Enterprise 2.0'

mapping.category_list = mapping.category_list.str.lower()

In [20]:
mapping.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 688 entries, 0 to 687
Data columns (total 10 columns):
category_list                              687 non-null object
Automotive & Sports                        688 non-null int64
Blanks                                     688 non-null int64
Cleantech / Semiconductors                 688 non-null int64
Entertainment                              688 non-null int64
Health                                     688 non-null int64
Manufacturing                              688 non-null int64
News, Search and Messaging                 688 non-null int64
Others                                     688 non-null int64
Social, Finance, Analytics, Advertising    688 non-null int64
dtypes: int64(9), object(1)
memory usage: 53.8+ KB


> > #### We now have all the tools in our arsenal to MAP the 'primary_sector' from master_frame1 to mapping['category_list']

In [21]:
#Converting the primary_sector to lower case inorder to map to mapping dataframe

master_frame1.primary_sector = master_frame1.primary_sector.str.lower()

## We are now creating our Master DataFrame i.e 'master_frame2' with all the DATA  

In [22]:
master_frame2 = pd.merge(master_frame1, mapping, left_on=master_frame1.primary_sector, right_on= mapping.category_list,how= 'left')

In [23]:
master_frame2.head(10)

,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd,permalink,name,homepage_url,category_list_x,...,category_list_y,Automotive & Sports,Blanks,Cleantech / Semiconductors,Entertainment,Health,Manufacturing,"News, Search and Messaging",Others,"Social, Finance, Analytics, Advertising"
0,/organization/-fame,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,05-01-2015,10000000.0,/organization/-fame,#fame,http://livfame.com,Media,...,media,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,/organization/-qounter,/funding-round/22dacff496eb7acb2b901dec1dfe5633,venture,A,14-10-2014,NaN,/organization/-qounter,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,...,application platforms,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,/organization/-qounter,/funding-round/b44fbb94153f6cdef13083530bb48030,seed,NaN,01-03-2014,700000.0,/organization/-qounter,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,...,application platforms,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,/organization/-the-one-of-them-inc-,/funding-round/650b8f704416801069bb178a1418776b,venture,B,30-01-2014,3406878.0,/organization/-the-one-of-them-inc-,"(THE) ONE of THEM,Inc.",http://oneofthem.jp,Apps|Games|Mobile,...,apps,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,/organization/0-6-com,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,19-03-2008,2000000.0,/organization/0-6-com,0-6.com,http://www.0-6.com,Curated Web,...,curated web,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5,/organization/004-technologies,/funding-round/1278dd4e6a37fa4b7d7e06c21b3c1830,venture,NaN,24-07-2014,NaN,/organization/004-technologies,004 Technologies,http://004gmbh.de/en/004-interact,Software,...,software,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
6,/organization/01games-technology,/funding-round/7d53696f2b4f607a2f2a8cbb83d01839,undisclosed,NaN,01-07-2014,41250.0,/organization/01games-technology,01Games Technology,http://www.01games.hk/,Games,...,games,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
7,/organization/0ndine-biomedical-inc,/funding-round/2b9d3ac293d5cdccbecff5c8cb0f327d,seed,NaN,11-09-2009,43360.0,/organization/0ndine-biomedical-inc,Ondine Biomedical Inc.,http://ondinebio.com,Biotechnology,...,biotechnology,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
8,/organization/0ndine-biomedical-inc,/funding-round/954b9499724b946ad8c396a57a5f3b72,venture,NaN,21-12-2009,719491.0,/organization/0ndine-biomedical-inc,Ondine Biomedical Inc.,http://ondinebio.com,Biotechnology,...,biotechnology,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
9,/organization/0xdata,/funding-round/383a9bd2c04f7038bb543ccef5ba3eae,seed,NaN,22-05-2013,3000000.0,/organization/0xdata,H2O.ai,http://h2o.ai/,Analytics,...,analytics,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [24]:
master_frame2.columns

Index(['company_permalink', 'funding_round_permalink', 'funding_round_type',
       'funding_round_code', 'funded_at', 'raised_amount_usd', 'permalink',
       'name', 'homepage_url', 'category_list_x', 'status', 'country_code',
       'state_code', 'region', 'city', 'founded_at', 'primary_sector',
       'category_list_y', 'Automotive & Sports', 'Blanks',
       'Cleantech / Semiconductors', 'Entertainment', 'Health',
       'Manufacturing', 'News, Search and Messaging', 'Others',
       'Social, Finance, Analytics, Advertising'],
      dtype='object')

#### There are few redundent columns which we can drop so that our master_frame2 is in good shape

In [25]:
master_frame2 = master_frame2.drop(['funding_round_permalink','category_list_x','category_list_y','Blanks'],axis='columns')

In [ ]:
master_frame2

In [26]:
master_frame2.shape

(114949, 23)

### _Creating dataframe d1 by filtering the TOP Country of Intrest i.e 'USA' and Funding Type i.e 'Venture' and making sure that funding amount is between 5 to 15 Million_

In [27]:
d1 = master_frame2.loc[(master_frame2.country_code == 'USA')]

d1 = d1.loc[(master_frame2.funding_round_type == 'venture')]

d1 = d1.loc[(d1.raised_amount_usd > 5.0e+6) & (d1.raised_amount_usd < 1.5e+7)]

d1['categorical_rep'] = (d1.iloc[:, -8:] == 1.0).idxmax(1)

d1 = d1.reset_index(drop= True)

In [28]:
d1.head()

,company_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd,permalink,name,homepage_url,status,country_code,...,primary_sector,Automotive & Sports,Cleantech / Semiconductors,Entertainment,Health,Manufacturing,"News, Search and Messaging",Others,"Social, Finance, Analytics, Advertising",categorical_rep
0,/organization/0xdata,venture,A,19-07-2014,8900000.0,/organization/0xdata,H2O.ai,http://h2o.ai/,operating,USA,...,analytics,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,"Social, Finance, Analytics, Advertising"
1,/organization/128-technology,venture,NaN,16-12-2014,11999347.0,/organization/128-technology,128 Technology,http://www.128technology.com/,operating,USA,...,service providers,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Others
2,/organization/1366-technologies,venture,A,03-02-2010,5150000.0,/organization/1366-technologies,1366 Technologies,http://www.1366tech.com,operating,USA,...,manufacturing,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Manufacturing
3,/organization/1366-technologies,venture,A,27-03-2008,12400000.0,/organization/1366-technologies,1366 Technologies,http://www.1366tech.com,operating,USA,...,manufacturing,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Manufacturing
4,/organization/1366-technologies,venture,B,16-12-2010,6000000.0,/organization/1366-technologies,1366 Technologies,http://www.1366tech.com,operating,USA,...,manufacturing,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Manufacturing


#### Selecting our main features for Analysis 

In [29]:
d1 = d1.loc[:,['primary_sector','categorical_rep','raised_amount_usd','company_permalink','country_code']]

In [30]:
d1.head()

,primary_sector,categorical_rep,raised_amount_usd,company_permalink,country_code
0,analytics,"Social, Finance, Analytics, Advertising",8900000.0,/organization/0xdata,USA
1,service providers,Others,11999347.0,/organization/128-technology,USA
2,manufacturing,Manufacturing,5150000.0,/organization/1366-technologies,USA
3,manufacturing,Manufacturing,12400000.0,/organization/1366-technologies,USA
4,manufacturing,Manufacturing,6000000.0,/organization/1366-technologies,USA


#### Count the number of times Investment is made in each of the 8 Main Sectors for USA between 5-15 million

In [31]:
d1_1 = d1.groupby('categorical_rep').count()

#sort count by descending order
d1_1 = d1_1.sort_values(by = 'primary_sector', ascending= False)
d1_1.primary_sector

categorical_rep
Others                                     2423
Social, Finance, Analytics, Advertising    2216
Cleantech / Semiconductors                 1991
News, Search and Messaging                 1281
Health                                      760
Manufacturing                               691
Entertainment                               479
Automotive & Sports                         143
Name: primary_sector, dtype: int64

#### Total no.of venture type investemts for USA in all 8 sectors between 5-15 million

In [32]:
#Total no.of venture type investemts for USA in all 8 sectors btw 5-15 million

np.sum(d1_1.primary_sector)

9984

#### Total sum of amount in venture type investemts for USA in all 8 sectors btw 5-15 million

In [33]:
d1_2 = d1.groupby('categorical_rep').sum()

np.sum(d1_2.raised_amount_usd)

89601347515.0

#### Company which raised the highest amount raised  based on the Top Sector investment wise i.e 'Others'

In [34]:
d1_3 = d1.loc[d1.categorical_rep == 'Others']

d1_3 = d1_3.groupby('company_permalink').sum()

d1_3.sort_values(by = 'raised_amount_usd',ascending= False).head()

,raised_amount_usd
company_permalink,
/organization/airtight-networks,54201907.0
/organization/bit9,48433533.0
/organization/zilliant,45600000.0
/organization/redfin,44800000.0
/organization/five9,44600000.0


#### Company which raised the highest amount raised  based on the Second Top Sector investment wise i.e 'Social, Finance, Analytics, Advertising'

In [35]:
d1_4 = d1.loc[d1.categorical_rep == 'Social, Finance, Analytics, Advertising']

d1_4 = d1_4.groupby('company_permalink').sum()

d1_4.sort_values(by = 'raised_amount_usd',ascending= False).head()

,raised_amount_usd
company_permalink,
/organization/shotspotter,67933006.0
/organization/firstrain,58344731.0
/organization/choicestream,58300000.0
/organization/netbase,55600000.0
/organization/chacha,53000000.0


### _Creating dataframe d2 by filtering the TOP Country of Intrest i.e 'GBR' and Funding Type i.e 'Venture' and making sure that funding amount is between 5 to 15 Million_

In [36]:
d2 = master_frame2.loc[(master_frame2.country_code == 'GBR')]

d2 = d2.loc[(master_frame2.funding_round_type == 'venture')]

d2 = d2.loc[(d2.raised_amount_usd > 5.0e+6) & (d2.raised_amount_usd < 1.5e+7)]

d2['categorical_rep'] = (d2.iloc[:, -8:] == 1.0).idxmax(1)

d2 = d2.reset_index(drop= True)

In [37]:
d2.head()

,company_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd,permalink,name,homepage_url,status,country_code,...,primary_sector,Automotive & Sports,Cleantech / Semiconductors,Entertainment,Health,Manufacturing,"News, Search and Messaging",Others,"Social, Finance, Analytics, Advertising",categorical_rep
0,/organization/365scores,venture,B,29-09-2014,5500000.0,/organization/365scores,365Scores,http://biz.365scores.com,operating,GBR,...,android,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,"Social, Finance, Analytics, Advertising"
1,/organization/7digital,venture,A,01-01-2008,8468328.0,/organization/7digital,7digital,http://about.7digital.com,acquired,GBR,...,content creators,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Entertainment
2,/organization/7digital,venture,NaN,19-10-2012,10000000.0,/organization/7digital,7digital,http://about.7digital.com,acquired,GBR,...,content creators,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Entertainment
3,/organization/90min,venture,NaN,07-05-2013,5800000.0,/organization/90min,90min,http://www.90min.com,operating,GBR,...,media,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Entertainment
4,/organization/abcodia,venture,B,18-05-2015,8259067.0,/organization/abcodia,Abcodia,http://abcodia.com,operating,GBR,...,biotechnology,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Cleantech / Semiconductors


#### Selecting our main features for Analysis 

In [38]:
d2 = d2.loc[:,['primary_sector','categorical_rep','raised_amount_usd','company_permalink','country_code']]

#### Count the number of times Investment is made in each of the 8 Main Sectors for GBR between 5-15 million

In [39]:
d2_1 = d2.groupby('categorical_rep').count()

#sort count by descending order
d2_1 = d2_1.sort_values(by = 'primary_sector', ascending= False)
d2_1.primary_sector

categorical_rep
Others                                     129
Cleantech / Semiconductors                 126
Social, Finance, Analytics, Advertising    118
News, Search and Messaging                  65
Entertainment                               48
Manufacturing                               38
Health                                      23
Automotive & Sports                         14
Name: primary_sector, dtype: int64

#### Total no.of venture type investemts for GBR in all 8 sectors between 5-15 million

In [40]:
#Total no.of venture type investemts for GBR in all 8 sectors btw 5-15 million

np.sum(d2_1.primary_sector)

561

#### Total sum of amount in venture type investemts for GBR in all 8 sectors btw 5-15 million

In [41]:
d2_2 = d2.groupby('categorical_rep').sum()

np.sum(d2_2.raised_amount_usd)

4941843539.0

#### Company which raised the highest amount raised  based on the Top Sector investment wise i.e 'Others'

In [42]:
d2_3 = d2.loc[d2.categorical_rep == 'Others']

d2_3 = d2_3.groupby('company_permalink').sum()

d2_3.sort_values(by = 'raised_amount_usd',ascending= False).head()

,raised_amount_usd
company_permalink,
/organization/electric-cloud,37000000.0
/organization/enigmatic,32500000.0
/organization/opencloud,27972766.0
/organization/myoptique-group,27694613.0
/organization/acs-clothing,26239014.0


#### Company which raised the highest amount raised  based on the Second Top Sector investment wise i.e 'Cleantech / Semiconductors'

In [43]:
d2_4 = d2.loc[d2.categorical_rep == 'Cleantech / Semiconductors']

d2_4 = d2_4.groupby('company_permalink').sum()

d2_4.sort_values(by = 'raised_amount_usd',ascending= False).head()

,raised_amount_usd
company_permalink,
/organization/eusa-pharma,35600000.0
/organization/horizon-discovery,31353719.0
/organization/f-star-biotechnologische-forschungs-und-entwicklungsges-m-b-h,29132906.0
/organization/takeda-cambridge,22340000.0
/organization/immune-targeting-systems,20320961.0


### _Creating dataframe d3 by filtering the TOP Country of Intrest i.e 'IND' and Funding Type i.e 'Venture' and making sure that funding amount is between 5 to 15 Million_

In [44]:
d3 = master_frame2.loc[(master_frame2.country_code == 'IND')]

d3 = d3.loc[(master_frame2.funding_round_type == 'venture')]

d3 = d3.loc[(d3.raised_amount_usd > 5.0e+6) & (d3.raised_amount_usd < 1.5e+7)]

d3['categorical_rep'] = (d3.iloc[:, -8:] == 1.0).idxmax(1)

d3 = d3.reset_index(drop= True)

In [45]:
d3.head()

,company_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd,permalink,name,homepage_url,status,country_code,...,primary_sector,Automotive & Sports,Cleantech / Semiconductors,Entertainment,Health,Manufacturing,"News, Search and Messaging",Others,"Social, Finance, Analytics, Advertising",categorical_rep
0,/organization/-fame,venture,B,05-01-2015,10000000.0,/organization/-fame,#fame,http://livfame.com,operating,IND,...,media,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Entertainment
1,/organization/21diamonds-india,venture,C,15-11-2012,6369507.0,/organization/21diamonds-india,21Diamonds,http://www.21diamonds.de,operating,IND,...,e-commerce,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Others
2,/organization/a-little-world,venture,B,09-09-2008,6410000.0,/organization/a-little-world,A LITTLE WORLD,http://alittleworld.com,operating,IND,...,finance,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,"Social, Finance, Analytics, Advertising"
3,/organization/adlabs-imagica,venture,NaN,28-10-2014,8180000.0,/organization/adlabs-imagica,Adlabs Imagica,http://www.adlabsimagica.com,operating,IND,...,entertainment,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Entertainment
4,/organization/agile,venture,A,01-05-2011,5740000.0,/organization/agile,Agile,http://www.agile-ft.com,operating,IND,...,finance,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,"Social, Finance, Analytics, Advertising"


#### Selecting our main features for Analysis 

In [46]:
d3 = d3.loc[:,['primary_sector','categorical_rep','raised_amount_usd','company_permalink','country_code']]

#### Count the number of times Investment is made in each of the 8 Main Sectors for IND between 5-15 million

In [47]:
d3_1 = d3.groupby('categorical_rep').count()

#sort count by descending order
d3_1 = d3_1.sort_values(by = 'primary_sector', ascending= False)
d3_1.primary_sector

categorical_rep
Others                                     78
Social, Finance, Analytics, Advertising    45
News, Search and Messaging                 41
Entertainment                              23
Manufacturing                              18
Health                                     17
Cleantech / Semiconductors                 14
Automotive & Sports                         7
Name: primary_sector, dtype: int64

#### Total no.of venture type investemts for IND in all 8 sectors between 5-15 million

In [48]:
#Total no.of venture type investemts for IND in all 8 sectors btw 5-15 million

np.sum(d3_1.primary_sector)

243

#### Total sum of amount in venture type investemts for IND in all 8 sectors btw 5-15 million

In [49]:
d3_2 = d3.groupby('categorical_rep').sum()

#Total sum of amount in venture type investemts for USA in all 8 sectors btw 5-15 million

np.sum(d3_2.raised_amount_usd)

2186543602.0

#### Company which raised the highest amount raised  based on the Top Sector investment wise i.e 'Others'

In [50]:
d3_3 = d3.loc[d3.categorical_rep == 'Others']

d3_3 = d3_3.groupby('company_permalink').sum()

d3_3.sort_values(by = 'raised_amount_usd',ascending= False).head()

,raised_amount_usd
company_permalink,
/organization/myntra,33000000.0
/organization/firstcry-com,24000000.0
/organization/nxtgen-data-center-cloud-services,22300000.0
/organization/maharana-infrastructure-and-professional-services-private-limited-mips,21600000.0
/organization/india-property-online,19000000.0


#### Company which raised the highest amount raised  based on the Top Sector investment wise i.e 'Social, Finance, Analytics, Advertising'

In [51]:
d3_4 = d3.loc[d3.categorical_rep == 'Social, Finance, Analytics, Advertising']

d3_4 = d3_4.groupby('company_permalink').sum()

d3_4.sort_values(by = 'raised_amount_usd',ascending= False).head()

,raised_amount_usd
company_permalink,
/organization/intarvo,21900000.0
/organization/grameen-financial-services,21556050.0
/organization/bankbazaar,19000000.0
/organization/microland,18300000.0
/organization/eka-software-solutions,16000000.0
